# Quant Backtesting Workflow in Marquee

This notebook demonstrates how to implement continuous portfolio optimizations using GS Quant.

## Step 1: Authenticate and Initialize your Session 

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt
import time
from math import copysign

import pandas as pd

from gs_quant.datetime.relative_date import RelativeDate
from gs_quant.markets.position_set import Position, PositionSet
from gs_quant.markets.portfolio import Portfolio
from gs_quant.markets.portfolio_manager import PortfolioManager
from gs_quant.markets.securities import Asset, AssetIdentifier
from gs_quant.markets.report import FactorRiskReport, ReturnFormat
from gs_quant.models.risk_model import FactorRiskModel
from gs_quant.session import GsSession
from gs_quant.target.common import PositionSetWeightingStrategy

from gs_quant.target.hedge import CorporateActionsTypes
from gs_quant.markets.optimizer import (
    OptimizerStrategy,
    OptimizerUniverse,
    AssetConstraint,
    FactorConstraint,
    SectorConstraint,
    OptimizerSettings,
    OptimizerConstraints,
    OptimizerObjective,
    OptimizerType,
)

pd.set_option('display.width', 1000)

client = None
secret = None

## External users must fill in their client ID and secret below and comment out the line below
# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(client_id=client, client_secret=secret)

print('GS Session initialized.')

### Quick Hint:
If you don't already have a portfolio set up, follow the instructions in  [01_Create_Backcasted_Portfolio](../Portfolios/01_Create_Backcasted_Portfolio.ipynb) to create a snapshot portfolio and backcast it. 
Alternatively, if you have a historical position set, you can also create a historical portfolio in [02_Create_New_Historical_Portfolio](../Portfolios/02_Create_New_Historical_Portfolio.ipynb)

## Step 2: Load your Portfolio


Load your portfolio and define hedge parameters

##### Note - The start date for your continuous optimization must be **after** the first position set uploaded in your source portfolio.

In [4]:
port_id = 'YOUR PORTFOLIO ID'  # put in your existing portfolio id
start_date = dt.date(2024, 1, 2)
hedge_notional_pct = 1.0
universe = ['SPX']
rebalance_freq = '1m'
risk_model_id = 'AXIOMA_AXWW4M'
apply_factor_constraints_on_total = True

In [ ]:
port_object = Portfolio.get(port_id)
port_manager = PortfolioManager(port_id)
print(f'Using portfolio {port_object.name} with id {port_id} as a source portfolio')

## Step 3: Create a Portfolio for your Optimization


You will first create a portfolio with a custom name and share it with a list of emails.


In [ ]:
opt_port_object = Portfolio(name=f'{port_object.name} - L/S Hedge')
opt_port_object.save()
opt_port_id = opt_port_object.id
print(f'Created portfolio {opt_port_object.name} with id {opt_port_id}')
opt_port_manager = PortfolioManager(opt_port_id)
opt_port_manager.share(['your.email@yourcompany.com', 'your.colleague@yourcompany.com'], admin=True)
opt_port_manager.set_tag_name_hierarchy(['source'])

opt_risk_report = FactorRiskReport(risk_model_id=risk_model_id)
opt_risk_report.set_position_source(opt_port_id)
opt_risk_report.save()

opt_port_manager.update_portfolio_tree()
print(f'Using portfolio {opt_port_object.name} with id {opt_port_id} as the optimization portfolio')

Risk and performance information about your source portfolio is required in order to run the continuous optimization. 

We'll first make sure that your source portfolio's factor risk and performance reports have finished successfully.

In [ ]:
print('Ensuring performance and risk calculations are complete on the source...')
perf_report = port_manager.get_performance_report()
perf_report.get_most_recent_job().wait_for_completion()

risk_report = port_manager.get_factor_risk_report(risk_model_id=risk_model_id)
risk_report.get_most_recent_job().wait_for_completion()

factor_exposure_data = risk_report.get_factor_exposure(start_date=start_date, end_date=start_date)
factor_exposure_map = factor_exposure_data.to_dict(orient='records')[0]

## Step 4: Setting Up Optimization Parameters

Now that you have a position set, you can get a hedge according to your liking.
`prepare_factor_constraints` below will pull constraints defined at a portfolio level to apply on the hedge.

In [8]:
def prepare_factor_constraints(factor_constraints, port_factor_exposure_map):
    """Given factor constraints defined on the total portfolio and factor exposures of the core portfolio,
    return constraints to be applied on the hedge"""
    new_constraints = []
    for fc in factor_constraints:
        old = fc.max_exposure
        new = port_factor_exposure_map.get(fc.factor.name, 0) - fc.max_exposure
        print('Changing factor constraint for ', fc.factor.name, 'from ', old, 'to ', new)
        new_constraints.append(
            FactorConstraint(fc.factor, port_factor_exposure_map.get(fc.factor.name, 0) - fc.max_exposure)
        )
    return new_constraints

Now that you have a position set, you can get a hedge according to your liking. We have put in some sample settings below. 

In [ ]:
hedge_universe = OptimizerUniverse(
    assets=[Asset.get(a, AssetIdentifier.BLOOMBERG_ID) for a in universe],
    explode_composites=True,
    exclude_corporate_actions_types=[CorporateActionsTypes.Mergers],
)

risk_model = FactorRiskModel.get(risk_model_id)

asset_constraints = [
    AssetConstraint(Asset.get('MSFT UW', AssetIdentifier.BLOOMBERG_ID), 0, 5),
    AssetConstraint(Asset.get('AAPL UW', AssetIdentifier.BLOOMBERG_ID), 0, 5),
]

# Specify the constraints on factor exposure of the Total Optimized Portfolio
factor_constraints = [
    FactorConstraint(risk_model.get_factor('Size'), 0),
]

if apply_factor_constraints_on_total:
    hedge_factor_constraints = prepare_factor_constraints(factor_constraints, factor_exposure_map)
else:
    hedge_factor_constraints = factor_constraints

sector_constraints = [SectorConstraint('Energy', 0, 30), SectorConstraint('Health Care', 0, 30)]
constraints = OptimizerConstraints(
    asset_constraints=asset_constraints,
    factor_constraints=hedge_factor_constraints,
    sector_constraints=sector_constraints,
)

## Step 5: Continuous Optimization Loop

With our initial setup done and settings configured, we are now ready to launch a flow that will continuously optimize our portfolio at our desired frequency.

We will take the positions from the previous rebalance, utilize performance analytics to get the latest positions, and then optimize the portfolio again. We will then update the portfolio with the new positions and schedule reports for the next rebalance date.

This operation of moving your portfolio forward using performance analytics relies solely on availability of the underlying assets. 

##### Note - The Optimizer is designed to work with prices adjusted for events such as dividends, splits, and corporate actions. This means it reflects the true economic value of assets after such adjustments. In contrast, our portfolio analytics operate on the raw (unadjusted) closing prices to provide a more detailed, day‐to‐day picture of historical trade activity. To avoid inconsistencies, we recommend passing position sets based solely on weight (which is independent of price adjustments) to derive the optimization output also in terms of weights

In [ ]:
port_manager = PortfolioManager(port_id)
start = start_date
max_end = RelativeDate("-1b", dt.date.today()).apply_rule(exchanges=['NYSE'])
start_time = time.time()
rebal = start_date

optimizer_runs = []

while rebal < max_end:
    print(f'Moving to rebalance date {rebal}')

    source_port_perf_report = port_manager.get_performance_report()
    latest_source_port_pos_data = source_port_perf_report.get_portfolio_constituents(
        start_date=rebal,
        end_date=rebal,
        fields=['quantity', 'grossWeight'],
        prefer_rebalance_positions=True,
        return_format=ReturnFormat.JSON,
    )
    latest_source_port_exp = source_port_perf_report.get_gross_exposure(start_date=rebal, end_date=rebal)[
        'grossExposure'
    ][0]
    latest_source_position_set = PositionSet(
        date=rebal,
        reference_notional=latest_source_port_exp,
        positions=[
            Position(
                asset_id=p['assetId'],
                identifier=p['assetId'],
                # We recommend using gross weight to find your reference weight, like below
                weight=copysign(p.get('grossWeight', 0), p.get('quantity', 0)),
                tags=[{'source': 'Portfolio'}],
            )
            for p in latest_source_port_pos_data
        ],
    )
    settings = OptimizerSettings(
        notional=hedge_notional_pct * latest_source_position_set.reference_notional,  # 40% of your original portfolio
        allow_long_short=True,
    )

    if factor_constraints and apply_factor_constraints_on_total:
        source_port_risk_report = port_manager.get_factor_risk_report(risk_model_id=risk_model_id)

        factor_exposure_data = risk_report.get_factor_exposure(
            start_date=latest_source_position_set.date, end_date=latest_source_position_set.date
        )
        factor_exposure_map = factor_exposure_data.to_dict(orient='records')[0]
        hedge_factor_constraints = prepare_factor_constraints(factor_constraints, factor_exposure_map)
    else:
        hedge_factor_constraints = factor_constraints

    constraints = OptimizerConstraints(
        asset_constraints=asset_constraints,
        factor_constraints=hedge_factor_constraints,
        sector_constraints=sector_constraints,
    )
    strategy = OptimizerStrategy(
        initial_position_set=latest_source_position_set,
        constraints=constraints,
        settings=settings,
        universe=hedge_universe,
        risk_model=risk_model,
        objective=OptimizerObjective.MINIMIZE_FACTOR_RISK,
    )
    print('Optimizing...')
    # Using adjusted prices in the Optimizer to reflect post-corporate action values
    strategy.run(optimizer_type=OptimizerType.AXIOMA_PORTFOLIO_OPTIMIZER)
    print('Optimization complete')
    optimizer_runs.append(strategy)
    optimization_result = strategy._OptimizerStrategy__result['hedge']
    optimization = PositionSet(
        date=strategy.initial_position_set.date,
        reference_notional=optimization_result['netExposure'],
        positions=[
            Position(identifier=asset.get('bbid', asset['name']), asset_id=asset['assetId'], weight=asset['weight'])
            for asset in optimization_result['constituents']
        ],
    )
    # Applying unadjusted close prices for portfolio analytics to preserve historical price granularity
    optimization.price(
        use_unadjusted_close_price=True, weighting_strategy=PositionSetWeightingStrategy.Weight, fallbackDate='5d'
    )
    for p in optimization.positions:
        p.add_tag('source', 'Optimization')
    # Again using unadjusted prices on source positions for consistency in analytics
    latest_source_position_set.price(
        use_unadjusted_close_price=True, weighting_strategy=PositionSetWeightingStrategy.Weight, fallbackDate='5d'
    )
    combined_pset = PositionSet(
        date=optimization.date,
        positions=[
            Position(identifier=p.identifier, asset_id=p.asset_id, quantity=p.quantity, tags=p.tags)
            for p in latest_source_position_set.positions + optimization.positions
        ],
    )
    opt_port_manager.update_positions([combined_pset])
    if not opt_port_manager.get_portfolio_tree().sub_portfolios:
        opt_port_manager.update_portfolio_tree()
    start = rebal
    rebal = min(max_end, RelativeDate(rebalance_freq, start).apply_rule())
    print(f'Scheduling reports to calculate performance till {rebal}...')
    opt_port_manager.schedule_reports(start_date=start, end_date=rebal)
    time.sleep(2)

print(f'Done! Processing completed in {time.time() - start_time} seconds')

print(f'As a reminder your optimization can be found in your "{opt_port_object.name}" portfolio with id {opt_port_id}')

You have successfully completed a basic run of our Quant Backtesting Workflow. 

For questions, please reach out to [Marquee Sales](mailto:gs-marquee-sales@gs.com)!